# Proposal for the Gammapy Modeling and Fitting API

This notebook outlines a proposal for restructuring and extending the current Gammapy modeling and fitting API to enable joint likelihood fitting. The proposal includes the introduction of new abstractions, represented by the following classes:

* `SkyModelIRF`: a "forward folded" model, that applies IRFs to a `SkyModel` instance and returns an integrated quantity corresponding to predicted counts. It can only be evaluated on a fixed grid, passed on input. It is basically, what the current `ModelEvaluator` does now, but with the model parameters attached.

* `BackgroundModel`: already integrated model, with fixed binning. It is initialized with a background map and introduces additional background parameters, such as `norm` or `tilt`. This model generic and not specific to 1D, 2D or 3D data.

* `NPredModel`: combines a list of `SkyModelIRF` and / or `BackgroundModel` and joins the parameters lists and sums up the contributions from all model components in the list. This model generic and not specific to 1D, 2D or 3D data.

* `Likelihood`: takes the binned data and model, updates the parameters and re-evaluates the model on every iteration of the fit. For different fit statistics sub-classes such as `CashLikelihood`, `WStatLikelihood` or `Chi2Likelihood` are introduced. This object is passed to the `Fit` object

* `JointLikelihood`: takes a list of `Likelihood` objects, joins the parameter lists and computes the joint likelihood. This object is passed to the `Fit` object.


Short term this code structure solves many uses cases without the introduction of `Datasets` and also replaces the current `MapFit`, as well as `FluxPointFit` and possibly `SpectrumFit` classes. Long-term we might re-introduce convenience classes, specific to certain tasks.

The use cases that can be solved with the proposed structure are listed below:

In [ ]:
from gammapy.cube import SkyModelIRF, BackgroundModel, NPredModel
from gammapy.utils.fitting import CashLikelihood, JointLikelihood

## Use Case 1: Simple Fit with Background Norm

Simple fit of a model to one observation, equivalent to what `MapFit` does now but with additional background parameters: 

In [ ]:
dnde_model = SkyModel()

# the exposure-map can have a different geometry than the background and exposure map, only after applying
# the PSF and Edisp it must be the same. This a natural place to introduce over oversampling for evaluating
# differential models
model = SkyModelIRF(exposure_map, psf, edisp, dnde_model)

background = BackgroundModel(background_map)

npred = NPredModel([model, background])
like = CashLikelihood(counts_map, npred)

fit = Fit(like)

fit.optimize()

## Use Case 2: Joint Fit across Multiple Observations

This use case involves a joint fit across mutiple observations with varying IRFs, but the same model. 

In [ ]:
dnde_model = SkyModel()

model_1 = SkyModelIRF(exposure_map_1, psf_1, edisp_1, dnde_model)
model_2 = SkyModelIRF(exposure_map_2, psf_2, edisp_2, dnde_model)


background_1 = BackgroundModel(background_map_1)
npred_1 = NPredModel([model_1, background_1])

background_2 = BackgroundModel(background_map_2)
npred_2 = NPredModel([model_2, background_2])


like_1 = CashLikelihood(counts_map_1, npred_1)
like_2 = CashLikelihood(counts_map_2, npred_2)


joint_like = JointLikelihood([like_1, like_2])

fit = Fit(joint_like)
fit.optimize()

## Use Case 3: IRFs per Source

In [ ]:
dnde_model_1 = SkyModel()
dnde_model_2 = SkyModel()


model_1 = SkyModelIRF(exposure_map, psf_1, edisp_1, dnde_model_1)
model_2 = SkyModelIRF(exposure_map, psf_2, edisp_2, dnde_model_2)

background = BackgroundModel(background_map)
npred = NPredModel([model_1, model_2, background])

like = CashLikelihood(counts_map, npred)

fit = Fit(like)
fit.optimize()

## Use Case 4: Joint Likelihood with Fermi

In [ ]:
dnde_model = SkyModel()
diffuse_model = SkyCubeDiffuse()

model = SkyModelIRF(exposure_map, psf, edisp, dnde_model)

model_fermi = SkyModelIRF(exposure_map_fermi, psf_fermi, edisp=None, dnde_model + diffuse_model)

# or alternatively precompute the diffuse emission as background model
fermi_diffuse = SkyModelIRF(exposure_map_fermi, psf_fermi, edisp=None, diffuse_model)
background_fermi = BackgroundModel(fermi_diffuse.evaluate())


background = BackgroundModel(background_map)
npred = NPredModel([model, background])

npred_fermi = NPredModel([model_fermi])

# or with the precomputed background
npred_fermi = NPredModel([model_fermi, background_fermi])


like = CashLikelihood(counts_map, npred)
like_fermi = CashLikelihood(counts_fermi, npred_fermi)

joint_like = JointLikelihood([like, like_fermi])

fit = Fit(joint_like)
fit.optimize()

# Use Case 5: Joint Likelihood with 3D and Flux Points

In [ ]:
spectral_model = SpectralModel()
spatial_model = SpatialModel()

dnde_model = SkyModel(spectral_model, spatial_model)

model = SkyModelIRF(exposure_map, psf, edisp, dnde_model)

background = BackgroundModel(background_map)
npred = NPredModel([model, background])

like = CashLikelihood(counts_map, npred)
like_fp = Chi2Likelihood(fp_data, spectral_model)

joint_like = JointLikelihood([like, like_fp])

fit = Fit(joint_like)
fit.optimize()

This is **just a first proposal** and of course there are many details to be sorted out. Open questions are:
* Where does the model.evaluate() call happen? Right now it is in the `Likelihood` class (see draft implementation below), does this make sense?
* How to handle masks during the likelihood evaluation and masks for joint-likelihoods (see draft implementation below)?
* Is the extra-abstraction of an `NPredModel` needed? 
        

In [ ]:
class Likelihood(object):
    """Binned likelihood object"""
    def __init__(data, model, mask=None):
        self.data = data
        self.model = model
        self.mask = mask
    
    # this method is the interface to the optimizer 
    # could also be named evaluate_fit() or similar
    def __call__(self, factors, mask=None):
        """Evaluate the likelihood for the fit"""
        self.model.parameters.set_factors(factors)      
        return self.total(mask)
    
    def per_bin(self):
        """Evaluate the likelihood per bin"""
        # IMPORTANT: it is not clear where the best place for the model evaluation call is.
        model = self.model.evaluate()
        likelihood = self.evaluate(self.data, model)
        return likelihood
    
    def total(self, mask=None):
        """Evaluate the total summed likelihood and apply optionally a mask"""        
        if self.mask:
            mask_default = self.mask.data
            if mask:
                # TODO: re-think how to combine the masks
                mask = mask & mask_default
            stat = self.per_bin[mask]
        else:
            stat = self.per_bin
        return np.sum(stat, dtype=np.float64)      

    
    
class CashLikelihood(Likelihood):
    """Cash statistics likelihood
    
    Parameters
    ----------
    counts : `Map`
        Counts map
    npred : `NPredModel`
        Predicted number of counts.
    mask : `Map`
        Mask to apply.
    """
    def __init__(self, counts=None, npred=None, mask=None):
        self.counts = counts
        self.npred = npred
        self.mask = mask   
        
    @property
    def model(self):
        """Alias for a nicer documentation"""
        return self.npred
    
    @property
    def data(self):
        """Alias for a nicer documentation"""
        return self.counts.data
    
    # This is a static method using pure numpy, because users might want
    # to call it directly CashLikelihood.evaluate(counts, npred), instead
    # of our current cash(counts, npred)
    @staticmethod
    def evaluate(counts, npred):
        """Cash likelihood formula"""
        with np.errstate(divide="ignore", invalid="ignore"):
            value = 2 * (npred - counts * np.log(npred))
            value = np.where(npred > 0, value, 0)
        return value
    
    
class Chi2Likelihood(Likelihood):        
    def per_bin(self):
        """Evaluate the likelihood per bin"""
        model = self.model.evaluate(parameters)
        
        likelihood = self.evaluate(self.data, model, model_err)
        return likelihood

    
    @statictmethod
    def evaluate(data, model, model_err):
        return ((data - model) / model_err) ** 2

    
class JointLikelihood(Likelihood):
    """Joint likelihood across mutiple likelihoods."""
    def __init__(self, likelihoods):
        """
        Parameters
        ----------
        likelihoods : list of `Likelihood` objects
            List of likelihoods to be combined.
        """
        self.likelihoods = likelihoods
    
    @property
    def model(self):
        """"""
        return ModelProxy()
        
    def per_bin(self):
        # As the binning can differ for different likelihoods this
        # does not make sense. Which in turn means you can't fit joint
        # likelihoods with optimizers, that rely on the per bin likelihood
        raise TypeError()
    
    def total(self, mask=None):
        """Total joint likelihood."""
        joint_likelihood = 0
        
        for like in self.likelihoods:
            joint_likelihood += like.total(mask)
            
        return joint_likelihood
       
    